# Final Data pipeline
***
Using dataset wthout any normalization and Feature engineering, Light Gradient Boosting Maching algorithm performed
really well on the Kaggle test dataset.
In the Pipeline using that model only.
***

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgbm
import joblib
import os
import seaborn as sns
import matplotlib.pyplot as plt
import neurokit2 as nk
from biosppy.signals import  ecg
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000
import warnings
warnings.filterwarnings("ignore")

In [83]:
def pipeline(testing=True, test_data=None, training=False, test_data_is_df=True):
    """
    This function train/test LGBM models
    parameters:
              testing(bool): True/False
              test_data(dataframe): pandas dataframe
              training(bool)L True/False
    Returns: None
    """
    if (testing) or training:
        d = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
        d_ = {0:'A',1:'B',2:'C',3:'D'}
        if training:
            print("\n<start>Loading dataset....")
            raw = pd.read_csv("train.csv")
            print("<complete>Loading dataset.\n")

            Y = raw['event']
            X = raw.drop('event',axis=1)

            X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=.2, shuffle=False)
            print(f"X_train.shape:{X_train.shape}")
            print(f"y_train.shape:{y_train.shape}")
            print(f"X_test.shape:{X_test.shape}")
            print(f"y_test.shape:{y_test.shape}")


            
            
            y_train_num = np.array(list(map(lambda x: d[x], y_train)))
            y_test_num =  np.array(list(map(lambda x: d[x], y_test)))

            X_train['experiment'] = X_train['experiment'].astype('category')
            X_test['experiment'] = X_test['experiment'].astype('category')



 
            train = lgbm.Dataset(X_train, label = y_train_num, categorical_feature=[1])
            test = lgbm.Dataset(X_test, label = y_test_num, categorical_feature=[1])

   
            params = {
                      "objective" : "multiclass", # used for multiclass softmax classifier
                      "metric" : "multi_error",   # Error rate for multiclass classification 
                      "boosting" :'gbdt',         # Using Gardient Boosted Decision Trees
                      'num_class':4,              # Number of desired output classes is 4 
                      "num_leaves" : 30,          # Number of leaves in Tree based algorithms
                      "learning_rate" : 0.01,     
                      "bagging_fraction" : 0.9,   # This is randomly select 90% of data without resampling\
                                                  #it will decrease impact of high variance on data
                      "bagging_seed" : 0,         # Random seeds for bagging
                      "num_threads" : 4,
                      "colsample_bytree" : 0.5,   # Subsampling fraction for feature
                      'min_data_in_leaf':100,     # Threshold on Data in a leaf
                      'min_split_gain':0.00019    # Minmimum gain threshold for splitting the node
            }

            print('\n<start>Training Light Gradient Boosting Machine...')
            model = lgbm.train(params, 
                              train_set = train,
                              num_boost_round=1000,
                              early_stopping_rounds=200,
                              verbose_eval=200, 
                              valid_sets=[train,test]
                              )
            print('<complete>Training\n') 
            
            
            print("\n<start>Prediction...")
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
            print("<complete>Prediction\n")


            print("\n<start>LOG-LOSS calculation...")
            print(f"Train log_loss:{log_loss(y_train, y_train_pred)}")
            print(f"Test log_loss:{log_loss(y_test, y_test_pred)}")
            print("<complete>LOG-LOSS calculation\n")

            conf_mat_val1 = confusion_matrix(np.argmax(y_train_pred, axis=1), y_train_num)
            conf_mat_val2 = confusion_matrix(np.argmax(y_test_pred, axis=1), y_test_num)
            
            
            ax = plt.axes()
            sns.heatmap(conf_mat_val1, annot=True,fmt="d",cmap='Blues')
            ax.set_xlabel('predicted', fontsize=10)
            ax.set_ylabel('actual', fontsize=10)
            ax.set_title("Train confusion matrix")
            ax.set_xticklabels(['A', 'B', 'C', 'D'])
            ax.set_yticklabels(['A', 'B', 'C', 'D'],rotation="horizontal")
            plt.show()
            
            ax = plt.axes()
            sns.heatmap(conf_mat_val2, annot=True,fmt="d",cmap='Blues')
            ax.set_xlabel('predicted', fontsize=10)
            ax.set_ylabel('actual', fontsize=10)
            ax.set_title("Test confusion matrix")
            ax.set_xticklabels(['A', 'B', 'C', 'D'])
            ax.set_yticklabels(['A', 'B', 'C', 'D'],rotation="horizontal")
            plt.show()
            
            
            print("====Dumping model====")
            joblib.dump(model, "model_final")
                
        if testing :
            print("\n<start>Testing on provided dataset...")
            model = joblib.load("model_final") if not training else model
            cols = ["id", 'crew', 'experiment', 'time', 'seat', 'eeg_fp1', 'eeg_f7', 'eeg_f8',
           'eeg_t4', 'eeg_t6', 'eeg_t5', 'eeg_t3', 'eeg_fp2', 'eeg_o1', 'eeg_p3',
           'eeg_pz', 'eeg_f3', 'eeg_fz', 'eeg_f4', 'eeg_c4', 'eeg_p4', 'eeg_poz',
           'eeg_c3', 'eeg_cz', 'eeg_o2', 'ecg', 'r', 'gsr']
            
            dtypes = ["int"]*2 + ["str"] + ["float"] + ["int"] + ["float"]*23
             
            if not test_data_is_df:
                if type(test_data) == type("s"):
                    col_dtype = list(zip(cols, dtypes)) 
                    myarray = np.array([test_data.split(",")])
                    record = np.array(list(map(tuple,myarray)),dtype=col_dtype)
                    test_data = pd.DataFrame.from_records(record)  
                else:
                    return print("<Error-3>Something went wrong...Returning!")
                print("Test data string being converted into test dataframe!")      
            
                
            if 'id' in test_data.columns:
                test_df = test_data.drop(['id'], axis=1 )
            if 'event' in test_data.columns:
                test_df = test_data.drop(['event'], axis=1 )
            test_df['experiment'] = test_df['experiment'].astype('category')
            test_pred = model.predict(test_df)
            pred_idx = np.argmax(test_pred, axis=-1)
            pred_classes = list(map(lambda x:d_[x], pred_idx))
            print(pred_classes[:10])
            print("<complete>Testing on provided dataset...\n")
        else:
            print("No testing?...fine...Returning...!")
          
    else:
        print("<Error-2>Something went wrong...Returning!")
        return 
    
    
        

In [87]:
pipeline(testing=True, test_data=t.loc[range(1000,1010)], training=False)
# pipeline(testing=True, test_data=test_str, training=False, test_data_is_df=False)


<start>Testing on provided dataset...
Test data string being converted into test dataframe!
['A']
<complete>Testing on provided dataset...



In [26]:
# test_str = "0,1,LOFT,0.0,0,17.8995,6.12783,0.994807,-28.2062,-47.695499,-187.080002,-33.183498,-4.22078,8.17816,33.160301000000004,33.8125,21.744699,16.2938,-7.04448,-14.4051,-4.0338400000000005,-0.393799,31.8381,17.0756,-8.13735,-7323.120117,643.1770019999999,594.778992"


##### Load data from here, if needed

In [6]:
# t = pd.read_csv('train.csv')
# tt = pd.read_csv("test.csv")

In [24]:
# dtypes = ["int64"]*2 + ["str"] + ["float64"] + ["int64"] + ["float64"]*23